In [101]:
import pandas as pd
import numpy as np
import os
import datetime as dt

### Load in both sheets and clean 2021 Data

In [102]:
csvpath21 = "202104-citibike-tripdata.csv"
csvpath20 = "202004-citibike-tripdata.csv"
april20 = pd.read_csv(csvpath20)
april20 = april20.reset_index(drop = True)
april21 = pd.read_csv(csvpath21)
april21 = april21.reset_index(drop = True)
april21.head()

C:\Users\emeyer79\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,7099F9531C9DA16D,docked_bike,2021-04-28 17:27:24,2021-04-28 17:34:31,DeKalb Ave & Hudson Ave,4513.06,Douglass St & 4 Ave,4175.14,40.689888,-73.981013,40.679279,-73.981540,member
1,EDC6F1B1A69A7863,docked_bike,2021-04-27 17:28:02,2021-04-27 17:33:07,E 47 St & Park Ave,6584.12,W 52 St & 6 Ave,6740.01,40.755102,-73.974986,40.761330,-73.979820,member
2,431A76830E10408A,docked_bike,2021-04-28 18:05:19,2021-04-28 18:10:48,E 47 St & Park Ave,6584.12,W 52 St & 6 Ave,6740.01,40.755102,-73.974986,40.761330,-73.979820,member
3,C00248A0ADC009FA,docked_bike,2021-04-20 21:35:47,2021-04-20 21:56:31,Irving Ave & DeKalb Ave,4898.02,Division Ave & Hooper St,5045.05,40.702700,-73.920950,40.706842,-73.954435,casual
4,BE78810D746EE516,docked_bike,2021-04-30 00:34:54,2021-04-30 01:03:29,3 Ave & E 112 St,7543.15,E 115 St & Lexington Ave,7599.09,40.795508,-73.941606,40.797911,-73.942300,casual


In [103]:
april21.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='object')

### Change start and end station ID to String

In [104]:
april21[['start_station_id', 'end_station_id']] = april21[['start_station_id', 'end_station_id']].astype(str)

### Look for dups

In [105]:
april21.ride_id.duplicated().sum()

0

In [106]:
april21.duplicated().sum()

0

In [107]:
april21.dtypes

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object

### Look at number of rows and columns and NaN find % missing

In [108]:
april21.shape

(2067669, 13)

In [109]:
april21_missing = april21.isna()
april21_missing.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False


In [110]:
april21_num_missing = april21_missing.sum()
april21_num_missing

ride_id                  0
rideable_type            0
started_at               0
ended_at                 0
start_station_name     110
start_station_id         0
end_station_name      3385
end_station_id           0
start_lat                0
start_lng                0
end_lat               3315
end_lng               3315
member_casual            0
dtype: int64

In [111]:
april21_num_missing/len(april21)

ride_id               0.000000
rideable_type         0.000000
started_at            0.000000
ended_at              0.000000
start_station_name    0.000053
start_station_id      0.000000
end_station_name      0.001637
end_station_id        0.000000
start_lat             0.000000
start_lng             0.000000
end_lat               0.001603
end_lng               0.001603
member_casual         0.000000
dtype: float64

In [112]:
april21.isna().mean().round(4)*100

ride_id               0.00
rideable_type         0.00
started_at            0.00
ended_at              0.00
start_station_name    0.01
start_station_id      0.00
end_station_name      0.16
end_station_id        0.00
start_lat             0.00
start_lng             0.00
end_lat               0.16
end_lng               0.16
member_casual         0.00
dtype: float64

In [113]:
april21_na = april21.dropna()
april21_na.shape

(2064277, 13)

In [114]:
april21.member_casual.unique()

array(['member', 'casual'], dtype=object)

### Add a Year column

In [115]:
april21_na['Year'] = '2021'
april21_na.columns


<ipython-input-115-071db97f5fdd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  april21_na['Year'] = '2021'


Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'Year'],
      dtype='object')

### Drop unwanted cols from 2021

In [116]:
april21_drop = april21_na.drop(['ride_id','rideable_type'],axis = 1)
april21_drop.columns

Index(['started_at', 'ended_at', 'start_station_name', 'start_station_id',
       'end_station_name', 'end_station_id', 'start_lat', 'start_lng',
       'end_lat', 'end_lng', 'member_casual', 'Year'],
      dtype='object')

### Separate Start/Stop Date and Time into new columns

In [117]:
april21_drop['started_at_date'] = pd.to_datetime(april21_drop['started_at']).dt.date
april21_drop['started_at_time'] = pd.to_datetime(april21_drop['started_at']).dt.time

In [118]:
april21_drop.head()

,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,Year,started_at_date,started_at_time
0,2021-04-28 17:27:24,2021-04-28 17:34:31,DeKalb Ave & Hudson Ave,4513.06,Douglass St & 4 Ave,4175.14,40.689888,-73.981013,40.679279,-73.981540,member,2021,2021-04-28,17:27:24
1,2021-04-27 17:28:02,2021-04-27 17:33:07,E 47 St & Park Ave,6584.12,W 52 St & 6 Ave,6740.01,40.755102,-73.974986,40.761330,-73.979820,member,2021,2021-04-27,17:28:02
2,2021-04-28 18:05:19,2021-04-28 18:10:48,E 47 St & Park Ave,6584.12,W 52 St & 6 Ave,6740.01,40.755102,-73.974986,40.761330,-73.979820,member,2021,2021-04-28,18:05:19
3,2021-04-20 21:35:47,2021-04-20 21:56:31,Irving Ave & DeKalb Ave,4898.02,Division Ave & Hooper St,5045.05,40.702700,-73.920950,40.706842,-73.954435,casual,2021,2021-04-20,21:35:47
4,2021-04-30 00:34:54,2021-04-30 01:03:29,3 Ave & E 112 St,7543.15,E 115 St & Lexington Ave,7599.09,40.795508,-73.941606,40.797911,-73.942300,casual,2021,2021-04-30,00:34:54


In [119]:
april21_drop['ended_at_date'] = pd.to_datetime(april21_drop['ended_at']).dt.date
april21_drop['ended_at_time'] = pd.to_datetime(april21_drop['ended_at']).dt.time
april21_drop.head()

,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,Year,started_at_date,started_at_time,ended_at_date,ended_at_time
0,2021-04-28 17:27:24,2021-04-28 17:34:31,DeKalb Ave & Hudson Ave,4513.06,Douglass St & 4 Ave,4175.14,40.689888,-73.981013,40.679279,-73.981540,member,2021,2021-04-28,17:27:24,2021-04-28,17:34:31
1,2021-04-27 17:28:02,2021-04-27 17:33:07,E 47 St & Park Ave,6584.12,W 52 St & 6 Ave,6740.01,40.755102,-73.974986,40.761330,-73.979820,member,2021,2021-04-27,17:28:02,2021-04-27,17:33:07
2,2021-04-28 18:05:19,2021-04-28 18:10:48,E 47 St & Park Ave,6584.12,W 52 St & 6 Ave,6740.01,40.755102,-73.974986,40.761330,-73.979820,member,2021,2021-04-28,18:05:19,2021-04-28,18:10:48
3,2021-04-20 21:35:47,2021-04-20 21:56:31,Irving Ave & DeKalb Ave,4898.02,Division Ave & Hooper St,5045.05,40.702700,-73.920950,40.706842,-73.954435,casual,2021,2021-04-20,21:35:47,2021-04-20,21:56:31
4,2021-04-30 00:34:54,2021-04-30 01:03:29,3 Ave & E 112 St,7543.15,E 115 St & Lexington Ave,7599.09,40.795508,-73.941606,40.797911,-73.942300,casual,2021,2021-04-30,00:34:54,2021-04-30,01:03:29


### Clean April 2020 Data

In [120]:
april20.columns

Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender'],
      dtype='object')

In [121]:
april21_drop.columns

Index(['started_at', 'ended_at', 'start_station_name', 'start_station_id',
       'end_station_name', 'end_station_id', 'start_lat', 'start_lng',
       'end_lat', 'end_lng', 'member_casual', 'Year', 'started_at_date',
       'started_at_time', 'ended_at_date', 'ended_at_time'],
      dtype='object')

In [122]:
april20.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,534,2020-04-01 00:00:15.2800,2020-04-01 00:09:09.8730,3656,E 2 St & Avenue A,40.723077,-73.985836,545,E 23 St & 1 Ave,40.736502,-73.978095,15162,Subscriber,1990,2
1,529,2020-04-01 00:02:28.9430,2020-04-01 00:11:18.3410,3163,Central Park West & W 68 St,40.773407,-73.977825,3301,Columbus Ave & W 95 St,40.791956,-73.968087,42861,Subscriber,1974,1
2,1488,2020-04-01 00:02:56.0900,2020-04-01 00:27:44.2530,3164,Columbus Ave & W 72 St,40.777057,-73.978985,3605,31 Ave & Crescent St,40.765835,-73.926547,35989,Subscriber,1982,1
3,341,2020-04-01 00:03:25.1410,2020-04-01 00:09:06.3190,3827,Halsey St & Broadway,40.685650,-73.915640,3869,Van Buren St & Broadway,40.692000,-73.926170,37548,Subscriber,1996,1
4,2206,2020-04-01 00:05:35.2790,2020-04-01 00:42:21.9080,3890,Grove St & Broadway,40.690470,-73.923511,3058,Lewis Ave & Kosciuszko St,40.692371,-73.937054,39701,Subscriber,1977,1


In [123]:
april21_drop.head()

,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,Year,started_at_date,started_at_time,ended_at_date,ended_at_time
0,2021-04-28 17:27:24,2021-04-28 17:34:31,DeKalb Ave & Hudson Ave,4513.06,Douglass St & 4 Ave,4175.14,40.689888,-73.981013,40.679279,-73.981540,member,2021,2021-04-28,17:27:24,2021-04-28,17:34:31
1,2021-04-27 17:28:02,2021-04-27 17:33:07,E 47 St & Park Ave,6584.12,W 52 St & 6 Ave,6740.01,40.755102,-73.974986,40.761330,-73.979820,member,2021,2021-04-27,17:28:02,2021-04-27,17:33:07
2,2021-04-28 18:05:19,2021-04-28 18:10:48,E 47 St & Park Ave,6584.12,W 52 St & 6 Ave,6740.01,40.755102,-73.974986,40.761330,-73.979820,member,2021,2021-04-28,18:05:19,2021-04-28,18:10:48
3,2021-04-20 21:35:47,2021-04-20 21:56:31,Irving Ave & DeKalb Ave,4898.02,Division Ave & Hooper St,5045.05,40.702700,-73.920950,40.706842,-73.954435,casual,2021,2021-04-20,21:35:47,2021-04-20,21:56:31
4,2021-04-30 00:34:54,2021-04-30 01:03:29,3 Ave & E 112 St,7543.15,E 115 St & Lexington Ave,7599.09,40.795508,-73.941606,40.797911,-73.942300,casual,2021,2021-04-30,00:34:54,2021-04-30,01:03:29


### Drop unwanted columns from 2020

In [124]:
april20_drop = april20.drop(['tripduration','bikeid','birth year','gender'],axis = 1)
april20_drop.columns

Index(['starttime', 'stoptime', 'start station id', 'start station name',
       'start station latitude', 'start station longitude', 'end station id',
       'end station name', 'end station latitude', 'end station longitude',
       'usertype'],
      dtype='object')

### Add Year column to 2020

In [125]:
april20_drop['Year'] = '2020'
april20_drop.columns

Index(['starttime', 'stoptime', 'start station id', 'start station name',
       'start station latitude', 'start station longitude', 'end station id',
       'end station name', 'end station latitude', 'end station longitude',
       'usertype', 'Year'],
      dtype='object')

### Find and Remove Nan

In [126]:
april20_drop.shape

(682762, 12)

In [127]:
april20_missing = april20_drop.isna()
april20_missing.shape

(682762, 12)

In [128]:
#No NaN in dataframe

april20_num_missing = april20_missing.sum()
april20_num_missing

starttime                  0
stoptime                   0
start station id           0
start station name         0
start station latitude     0
start station longitude    0
end station id             0
end station name           0
end station latitude       0
end station longitude      0
usertype                   0
Year                       0
dtype: int64

### Separate Date Time columnd for 2020

In [156]:
april20_drop['started_at_date'] = pd.to_datetime(april20_drop['starttime']).dt.date
april20_drop['started_at_time'] = pd.to_datetime(april20_drop['starttime']).dt.time
april20_drop.head()

,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,usertype,Year,started_at_date,started_at_time
0,2020-04-01 00:00:15.2800,2020-04-01 00:09:09.8730,3656,E 2 St & Avenue A,40.723077,-73.985836,545,E 23 St & 1 Ave,40.736502,-73.978095,Subscriber,2020,2020-04-01,00:00:15.280000
1,2020-04-01 00:02:28.9430,2020-04-01 00:11:18.3410,3163,Central Park West & W 68 St,40.773407,-73.977825,3301,Columbus Ave & W 95 St,40.791956,-73.968087,Subscriber,2020,2020-04-01,00:02:28.943000
2,2020-04-01 00:02:56.0900,2020-04-01 00:27:44.2530,3164,Columbus Ave & W 72 St,40.777057,-73.978985,3605,31 Ave & Crescent St,40.765835,-73.926547,Subscriber,2020,2020-04-01,00:02:56.090000
3,2020-04-01 00:03:25.1410,2020-04-01 00:09:06.3190,3827,Halsey St & Broadway,40.685650,-73.915640,3869,Van Buren St & Broadway,40.692000,-73.926170,Subscriber,2020,2020-04-01,00:03:25.141000
4,2020-04-01 00:05:35.2790,2020-04-01 00:42:21.9080,3890,Grove St & Broadway,40.690470,-73.923511,3058,Lewis Ave & Kosciuszko St,40.692371,-73.937054,Subscriber,2020,2020-04-01,00:05:35.279000


In [157]:
april20_drop['ended_at_date'] = pd.to_datetime(april20_drop['stoptime']).dt.date
april20_drop['ended_at_time'] = pd.to_datetime(april20_drop['stoptime']).dt.time
april20_drop.head()

,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,usertype,Year,started_at_date,started_at_time,ended_at_date,ended_at_time
0,2020-04-01 00:00:15.2800,2020-04-01 00:09:09.8730,3656,E 2 St & Avenue A,40.723077,-73.985836,545,E 23 St & 1 Ave,40.736502,-73.978095,Subscriber,2020,2020-04-01,00:00:15.280000,2020-04-01,00:09:09.873000
1,2020-04-01 00:02:28.9430,2020-04-01 00:11:18.3410,3163,Central Park West & W 68 St,40.773407,-73.977825,3301,Columbus Ave & W 95 St,40.791956,-73.968087,Subscriber,2020,2020-04-01,00:02:28.943000,2020-04-01,00:11:18.341000
2,2020-04-01 00:02:56.0900,2020-04-01 00:27:44.2530,3164,Columbus Ave & W 72 St,40.777057,-73.978985,3605,31 Ave & Crescent St,40.765835,-73.926547,Subscriber,2020,2020-04-01,00:02:56.090000,2020-04-01,00:27:44.253000
3,2020-04-01 00:03:25.1410,2020-04-01 00:09:06.3190,3827,Halsey St & Broadway,40.685650,-73.915640,3869,Van Buren St & Broadway,40.692000,-73.926170,Subscriber,2020,2020-04-01,00:03:25.141000,2020-04-01,00:09:06.319000
4,2020-04-01 00:05:35.2790,2020-04-01 00:42:21.9080,3890,Grove St & Broadway,40.690470,-73.923511,3058,Lewis Ave & Kosciuszko St,40.692371,-73.937054,Subscriber,2020,2020-04-01,00:05:35.279000,2020-04-01,00:42:21.908000


In [158]:
april20_drop.columns

Index(['starttime', 'stoptime', 'start station id', 'start station name',
       'start station latitude', 'start station longitude', 'end station id',
       'end station name', 'end station latitude', 'end station longitude',
       'usertype', 'Year', 'started_at_date', 'started_at_time',
       'ended_at_date', 'ended_at_time'],
      dtype='object')

In [159]:
april21_drop.columns

Index(['started_at', 'ended_at', 'start_station_name', 'start_station_id',
       'end_station_name', 'end_station_id', 'start_lat', 'start_lng',
       'end_lat', 'end_lng', 'member_casual', 'Year', 'started_at_date',
       'started_at_time', 'ended_at_date', 'ended_at_time'],
      dtype='object')

In [160]:
april20_rename = april20_drop.rename(columns={"starttime": "started_at", "stoptime": "ended_at", "start station id":"start_station_id",
                            "end station id":"end_station_id", "start station name":"start_station_name",
                            "end station name":"end_station_name", "start station latitude":"start_lat", 
                            "start station longitude":"start_lng", "end station latitude":"end_lat", "end station longitude":
                            "end_lng", "usertype":"member_casual"})
april20_rename.columns


Index(['started_at', 'ended_at', 'start_station_id', 'start_station_name',
       'start_lat', 'start_lng', 'end_station_id', 'end_station_name',
       'end_lat', 'end_lng', 'member_casual', 'Year', 'started_at_date',
       'started_at_time', 'ended_at_date', 'ended_at_time'],
      dtype='object')

### Rearrange 2020 columns to reflect 2021


In [161]:
april20_rename.shape

(682762, 16)

In [162]:
april21_drop.shape

(2064277, 16)

In [163]:
april20_cols = april20_rename[['started_at', 'ended_at', 'start_station_name', 'start_station_id',
       'end_station_name', 'end_station_id', 'start_lat', 'start_lng',
       'end_lat', 'end_lng', 'member_casual', 'Year', 'started_at_date',
       'started_at_time', 'ended_at_date', 'ended_at_time']]
april20_cols.columns

Index(['started_at', 'ended_at', 'start_station_name', 'start_station_id',
       'end_station_name', 'end_station_id', 'start_lat', 'start_lng',
       'end_lat', 'end_lng', 'member_casual', 'Year', 'started_at_date',
       'started_at_time', 'ended_at_date', 'ended_at_time'],
      dtype='object')

In [164]:
april21_drop.columns

Index(['started_at', 'ended_at', 'start_station_name', 'start_station_id',
       'end_station_name', 'end_station_id', 'start_lat', 'start_lng',
       'end_lat', 'end_lng', 'member_casual', 'Year', 'started_at_date',
       'started_at_time', 'ended_at_date', 'ended_at_time'],
      dtype='object')

In [165]:
april21_drop.member_casual.unique()

array(['member', 'casual'], dtype=object)

In [166]:
april20_cols.member_casual.unique()

array(['Subscriber', 'Customer'], dtype=object)

In [170]:
april20_cols['member_casual'] = april20_cols['member_casual'].replace(['Subscriber'], 'member')
april20_cols.member_casual.unique()

array(['member', 'Customer'], dtype=object)

In [171]:
april20_cols['member_casual'] = april20_cols['member_casual'].replace(['Customer'], 'casual')
april20_cols.member_casual.unique()

array(['member', 'casual'], dtype=object)

In [172]:
april20_cols.head()

,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,Year,started_at_date,started_at_time,ended_at_date,ended_at_time
0,2020-04-01 00:00:15.2800,2020-04-01 00:09:09.8730,E 2 St & Avenue A,3656,E 23 St & 1 Ave,545,40.723077,-73.985836,40.736502,-73.978095,member,2020,2020-04-01,00:00:15.280000,2020-04-01,00:09:09.873000
1,2020-04-01 00:02:28.9430,2020-04-01 00:11:18.3410,Central Park West & W 68 St,3163,Columbus Ave & W 95 St,3301,40.773407,-73.977825,40.791956,-73.968087,member,2020,2020-04-01,00:02:28.943000,2020-04-01,00:11:18.341000
2,2020-04-01 00:02:56.0900,2020-04-01 00:27:44.2530,Columbus Ave & W 72 St,3164,31 Ave & Crescent St,3605,40.777057,-73.978985,40.765835,-73.926547,member,2020,2020-04-01,00:02:56.090000,2020-04-01,00:27:44.253000
3,2020-04-01 00:03:25.1410,2020-04-01 00:09:06.3190,Halsey St & Broadway,3827,Van Buren St & Broadway,3869,40.685650,-73.915640,40.692000,-73.926170,member,2020,2020-04-01,00:03:25.141000,2020-04-01,00:09:06.319000
4,2020-04-01 00:05:35.2790,2020-04-01 00:42:21.9080,Grove St & Broadway,3890,Lewis Ave & Kosciuszko St,3058,40.690470,-73.923511,40.692371,-73.937054,member,2020,2020-04-01,00:05:35.279000,2020-04-01,00:42:21.908000


In [173]:
frames = [april20_cols, april21_drop]
merged = pd.concat(frames)

In [175]:
merged.shape

(2747039, 16)

In [176]:
april20_cols.shape

(682762, 16)

In [177]:
april21_drop.shape

(2064277, 16)

In [178]:
merged.reset_index(drop=True)

,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,Year,started_at_date,started_at_time,ended_at_date,ended_at_time
0,2020-04-01 00:00:15.2800,2020-04-01 00:09:09.8730,E 2 St & Avenue A,3656,E 23 St & 1 Ave,545,40.723077,-73.985836,40.736502,-73.978095,member,2020,2020-04-01,00:00:15.280000,2020-04-01,00:09:09.873000
1,2020-04-01 00:02:28.9430,2020-04-01 00:11:18.3410,Central Park West & W 68 St,3163,Columbus Ave & W 95 St,3301,40.773407,-73.977825,40.791956,-73.968087,member,2020,2020-04-01,00:02:28.943000,2020-04-01,00:11:18.341000
2,2020-04-01 00:02:56.0900,2020-04-01 00:27:44.2530,Columbus Ave & W 72 St,3164,31 Ave & Crescent St,3605,40.777057,-73.978985,40.765835,-73.926547,member,2020,2020-04-01,00:02:56.090000,2020-04-01,00:27:44.253000
3,2020-04-01 00:03:25.1410,2020-04-01 00:09:06.3190,Halsey St & Broadway,3827,Van Buren St & Broadway,3869,40.685650,-73.915640,40.692000,-73.926170,member,2020,2020-04-01,00:03:25.141000,2020-04-01,00:09:06.319000
4,2020-04-01 00:05:35.2790,2020-04-01 00:42:21.9080,Grove St & Broadway,3890,Lewis Ave & Kosciuszko St,3058,40.690470,-73.923511,40.692371,-73.937054,member,2020,2020-04-01,00:05:35.279000,2020-04-01,00:42:21.908000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2747034,2021-04-18 15:48:21,2021-04-18 16:03:42,Hudson St & W 13 St,6115.06,Mercer St & Spring St,5532.01,40.740057,-74.005274,40.723627,-73.999496,casual,2021,2021-04-18,15:48:21,2021-04-18,16:03:42
2747035,2021-04-19 19:39:00,2021-04-19 19:55:18,W 87 St & Amsterdam Ave,7458.03,3 Ave & E 62 St,6762.04,40.788390,-73.974700,40.763126,-73.965269,member,2021,2021-04-19,19:39:00,2021-04-19,19:55:18
2747036,2021-04-21 11:57:03,2021-04-21 12:15:49,E 27 St & 1 Ave,6004.06,3 Ave & E 62 St,6762.04,40.739445,-73.976806,40.763126,-73.965269,member,2021,2021-04-21,11:57:03,2021-04-21,12:15:49
2747037,2021-04-04 14:41:23,2021-04-04 14:56:46,Myrtle Ave & St Edwards St,4659.02,Brooklyn Bridge Park - Pier 2,4756.04,40.693270,-73.977038,40.698458,-73.997178,member,2021,2021-04-04,14:41:23,2021-04-04,14:56:46


In [179]:
merged.to_csv(r'C:\Users\emeyer79\Desktop\Tableau HW\merged_final.csv', index = False)
